# Logistic Regression Projesi


**Kodlar:**
1. Veri Ön İşleme (Preprocessing)
2. Scikit-Learn ile Logistic Regression Modeli
3. Custom Logistic Regression Modeli (Optimizasyonlar ile)

In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def preprocess_data(input_file: str, output_file: str = None) -> pd.DataFrame:
    """
    Excel dosyasından Titanic veri setini okuyup; gereksiz sütunları kaldırır, eksik değerleri doldurur, 
    kategorik sütunları dummy değişkenlere dönüştürür ve numerik sütunları ölçeklendirir.
    İşlenmiş DataFrame'i döndürür ve istenirse dosyaya kaydeder.
    """
    df = pd.read_excel(input_file)
    
    print("Orijinal veri seti sütunları:")
    print(df.columns.tolist())
    print("\nOrijinal veri seti eksik değerler:")
    print(df.isnull().sum())
    print("\nOrijinal veri setinin ilk 5 satırı:")
    print(df.head())
    
    # Gereksiz sütunların çıkarılması
    columns_to_drop = ['Cabin', 'Name', 'Ticket', 'PassengerId']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
    print("\nGereksiz sütunlar çıkarıldıktan sonra sütunlar:")
    print(df.columns.tolist())
    
    # Numerik sütunlarda eksik değerlerin doldurulması
    if 'Age' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age'].median())
    if 'Fare' in df.columns:
        df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    print("\nNumerik sütunlarda eksik değerler doldurulduktan sonra (Age, Fare):")
    print(df[['Age', 'Fare']].isnull().sum())
    
    # Kategorik sütunlarda eksik değerlerin doldurulması
    if 'Embarked' in df.columns:
        df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    for col in df.select_dtypes(include='object').columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mode()[0])
    print("\nKategorik sütunlarda eksik değerler doldurulduktan sonra:")
    print(df.select_dtypes(include='object').isnull().sum())
    
    # Kategorik değişkenlerin dummy değişkenlere dönüştürülmesi
    categorical_cols = ['Sex', 'Embarked']
    for col in categorical_cols:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
            df = pd.concat([df, dummies], axis=1)
            df.drop(col, axis=1, inplace=True)
    print("\nKategorik değişkenler dummy değişkenlere dönüştürüldükten sonra sütunlar:")
    print(df.columns.tolist())
    
    # Numerik sütunların ölçeklenmesi
    scaler = StandardScaler()
    for col in ['Age', 'Fare']:
        if col in df.columns:
            df[col] = scaler.fit_transform(df[[col]])
    print("\nNumerik sütunlar ölçeklendirildikten sonra (Age, Fare) tanımlayıcı istatistikler:")
    print(df[['Age', 'Fare']].describe())
    
    print("\nİşlenmiş veri setinin ilk 5 satırı:")
    print(df.head())
    
    if output_file:
        df.to_excel(output_file, index=False)
        print(f"\nİşlenmiş veri seti {output_file} dosyasına kaydedildi.")
    
    return df

# Preprocessing'i çalıştır
df_processed = preprocess_data("Titanic-Dataset.xlsx", "Titanic-Dataset-Processed.xlsx")

Orijinal veri seti sütunları:
['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

Orijinal veri seti eksik değerler:
PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

Orijinal veri setinin ilk 5 satırı:
   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0            1         0       3    male  22.0      1      0   7.2500        S
1            2         1       1  female  38.0      1      0  71.2833        C
2            3         1       3  female  26.0      0      0   7.9250        S
3            4         1       1  female  35.0      1      0  53.1000        S
4            5         0       3    male  35.0      0      0   8.0500        S

Gereksiz sütunlar çıkarıldıktan sonra sütunlar:
['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

Numerik sütunlarda eksik

## Scikit-Learn ile Logistic Regression

Aşağıdaki kod, işlenmiş veri setini kullanarak Scikit-Learn'ün `LogisticRegression` sınıfı ile modeli eğitir, tahmin yapar ve sonuçları değerlendirir.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import time

def run_scikit_lr(df):
    X = df.drop("Survived", axis=1)
    y = df["Survived"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression(max_iter=1000)
    
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    start_time = time.time()
    y_pred = model.predict(X_test)
    prediction_time = time.time() - start_time
    
    print("Training time: {:.4f} seconds".format(training_time))
    print("Prediction time: {:.4f} seconds".format(prediction_time))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

# Scikit-Learn modelini çalıştır
run_scikit_lr(df_processed)

Training time: 0.0041 seconds
Prediction time: 0.0007 seconds

Confusion Matrix:
[[90 15]
 [19 55]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



## Custom Logistic Regression

Aşağıdaki kod custom Logistic Regression modelini gradient descent kullanarak eğitir.
Bu modelde erken durdurma (early stopping) ve L2 regularizasyon vardır

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import time

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function(theta, X, y, reg_lambda=0):
    m = len(y)
    h = sigmoid(X.dot(theta))
    epsilon = 1e-5
    cost = - (1/m) * np.sum(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon))
    # L2 regularizasyon (bias hariç)
    reg_term = (reg_lambda / (2*m)) * np.sum(theta[1:] ** 2)
    cost += reg_term
    grad = (1/m) * X.T.dot(h - y)
    grad[1:] += (reg_lambda / m) * theta[1:]
    return cost, grad

def gradient_descent(X, y, theta, learning_rate, iterations, tol=1e-6, reg_lambda=0):
    cost_history = []
    for i in range(iterations):
        cost, grad = cost_function(theta, X, y, reg_lambda)
        theta = theta - learning_rate * grad
        cost_history.append(cost)
        if i % 100 == 0:
            print(f"Iteration {i}: Cost = {cost}")
        if i > 0 and abs(cost_history[-2] - cost_history[-1]) < tol:
            print(f"Convergence reached at iteration {i}")
            break
    return theta, cost_history

def run_custom_lr(df):
    print("\n----- Custom Logistic Regression -----")
    # Tüm sütunları numeric (float) tipinde olduğundan emin olarak NumPy array'e dönüştür
    X_np = df.drop("Survived", axis=1).astype(float).values
    y_np = df["Survived"].astype(float).values
    
    # Bias terimini ekle
    X_np = np.hstack([np.ones((X_np.shape[0], 1)), X_np])
    
    X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_np, y_np, test_size=0.2, random_state=42)
    
    theta = np.zeros(X_train_np.shape[1])
    learning_rate = 0.01
    iterations = 1000
    reg_lambda = 0.1  # L2 regularizasyon parametresi
    
    start_time = time.time()
    theta, cost_history = gradient_descent(X_train_np, y_train_np, theta, learning_rate, iterations, tol=1e-6, reg_lambda=reg_lambda)
    training_time_custom = time.time() - start_time
    
    print("\nCustom model final cost:", cost_history[-1])
    print("Custom model training time: {:.4f} seconds".format(training_time_custom))
    
    def predict(theta, X):
        return (sigmoid(X.dot(theta)) >= 0.5).astype(int)
    
    start_time = time.time()
    y_pred_custom = predict(theta, X_test_np)
    prediction_time_custom = time.time() - start_time
    
    print("Custom model prediction time: {:.4f} seconds".format(prediction_time_custom))
    print("\nCustom model Confusion Matrix:")
    print(confusion_matrix(y_test_np, y_pred_custom))
    print("\nCustom model Classification Report:")
    print(classification_report(y_test_np, y_pred_custom))

# Custom modeli çalıştır
run_custom_lr(df_processed)


----- Custom Logistic Regression -----
Iteration 0: Cost = 0.6931271807599426
Iteration 100: Cost = 0.6141685388556968
Iteration 200: Cost = 0.5979830977239932
Iteration 300: Cost = 0.5863592372088651
Iteration 400: Cost = 0.5768961394274994
Iteration 500: Cost = 0.5688638553207505
Iteration 600: Cost = 0.5618711666674641
Iteration 700: Cost = 0.5556813364941928
Iteration 800: Cost = 0.5501383662838315
Iteration 900: Cost = 0.5451321637809293

Custom model final cost: 0.540624327638365
Custom model training time: 0.0287 seconds
Custom model prediction time: 0.0000 seconds

Custom model Confusion Matrix:
[[103   2]
 [ 47  27]]

Custom model Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.98      0.81       105
         1.0       0.93      0.36      0.52        74

    accuracy                           0.73       179
   macro avg       0.81      0.67      0.67       179
weighted avg       0.79      0.73      0.69       179



## Sonuç

Bu notebook, veri ön işleme, Scikit-Learn Logistic Regression ve custom (optimizasyonlu) Logistic Regression model eğitim adımlarını içerir.
Eğitim ve tahmin süreleri, karmaşıklık matrisi ve sınıflandırma raporları çıktı olarak gösterilecektir.